In [3]:
!pip install openai tiktoken langchain huggingface_hub google-search-results faiss-cpu > /dev/null

In [2]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.experimental import BabyAGI

In [4]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [21]:
import os
import faiss
os.environ['OPENAI_API_KEY']=''
os.environ['SERPAPI_API_KEY']=''

In [11]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()

embedding_size = 1536

index = faiss.IndexFlatL2(embedding_size)

vectorstore = FAISS(embeddings_model.embed_query, 
                    index, 
                    InMemoryDocstore({}), {})


In [12]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)

In [15]:
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

search = SerpAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]

In [16]:
prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""

suffix = """Question: {task}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [17]:
llm = OpenAI(temperature=0)

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(llm_chain=llm_chain, 
                      allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [18]:
babyagi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=False,
    max_iterations=1
)

In [19]:
objective = """Write report on the current stockmarket status"""

In [20]:
babyagi({"objective": objective})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to come up with a todo list for the current stockmarket status
Action: TODO
Action Input: Create a todo list for the current stockmarket status
Observation: 

1. Research current stock market trends and news
2. Analyze current stock market performance
3. Identify stocks that are performing well
4. Identify stocks that are underperforming
5. Research the fundamentals of stocks that are performing well
6. Research the fundamentals of stocks that are underperforming
7. Identify potential investment opportunities
8. Analyze risk/reward of potential investments
9. Create a portfolio of stocks to invest in
10. Monitor stock market performance and adjust portfolio accordingly
Thought: I now know the final answer
Final Answer: The todo list for the current stockmarket status is to research current stock market trends and news, analyze current stock market 

{'objective': 'Write report on the current stockmarket status'}